In [6]:
pip install cvxpy


In [7]:
pip install dccp

In [8]:
import pandas as pd
import numpy as np
import random
import dccp

from cvxpy import *
import cvxpy as cvx

import tensorflow as tf
import keras

from keras.layers import Dense, Input
from tensorflow.keras import Model

from sklearn.metrics import confusion_matrix

In [9]:
## data preprocessing

raw = pd.read_csv("./compas-scores-two-years.csv")

## feature
name_list1 = [ 'sex','age', 'race', 'priors_count',
             'c_charge_degree', 'c_charge_desc',
             'start', 'end', 'event', 'two_year_recid'] # 11


raw_data_for_train = raw.loc[:,name_list1]

print(raw_data_for_train.shape) 

# dropna 7214 -> 7185
raw_data_for_train = raw_data_for_train.dropna()
print(raw_data_for_train.shape)

print("#"*100)
print("#"*100)
########################################
########################################

## Reorder training data
African_American_index = raw_data_for_train.race == 'African-American'
Caucasian_index = raw_data_for_train.race == 'Caucasian'
print(raw_data_for_train[African_American_index].shape,raw_data_for_train[Caucasian_index].shape)

AA_train = raw_data_for_train[African_American_index]
CA_train = raw_data_for_train[Caucasian_index]
train_data = pd.concat( (AA_train, CA_train) ) 

'''
note
train_data is a df with dimension (6129 rows × 10 columns) 
first (3684,10) is z/s = 0
last (2445,10) is z/s = 1
now race is still in categorical form
'''
print("#"*100)
print("#"*100)
########################################
########################################


## label
train_label = train_data.two_year_recid
del train_data["two_year_recid"]  
print(train_label.shape)

## feature
# 1. normalizing numerical features
numeric_features = train_data.dtypes[train_data.dtypes != 'object'].index
train_data[numeric_features] = train_data[numeric_features].apply(lambda x: (x - x.mean()) / (x.std()))

# 2. convert sensitive feature to numerical feature 
# race = "African-American"/"Caucasian"  -->  race_num = 0/1
train_data = train_data.assign(race_num = (train_data['race']!= 'African-American') | (train_data['race']== 'Caucasian') ) # warning! + -> |
train_data.race_num = train_data.race_num.astype('int64')
del train_data["race"]
print(train_data.race_num[:10],"\n",train_data.race_num[-10:], train_data.race_num[:3684].mean(),train_data.race_num[3684:].mean())

# 3. convert other categorical features to dummy variable (one_hot)
all_features = pd.get_dummies(train_data, dummy_na=True) 

'''
note
print(all_features.shape) will return (6129, 423)
'''

print("#"*100)
print("#"*100)
########################################
########################################


## Put sensitive variable i.e. race_num to last column
reorder_column_name = list(all_features.keys())
# print(len(reorder_column_name))
race_num_name = reorder_column_name.pop(5)
# print(race_num_name,len(reorder_column_name))
reorder_column_name.append(race_num_name)
# print(reorder_column_name[-3:],len(reorder_column_name))
all_features = all_features[reorder_column_name]
print("test")
print(all_features.iloc[:,-2:])

'''
note

3 print will return
423
race_num 422
['c_charge_desc_arrest case no charge', 'c_charge_desc_nan', 'race_num'] 423

'''

## create and shuffle train test validation (5:1:1)
num_examples = all_features.shape[0]
# print(num_examples)
indices = list(range(num_examples))
random.shuffle(indices)
# print(np.max(indices))

all_features_shuffled = all_features.values[indices]
train_label_shuffled = train_label.values[indices]

x_train = all_features_shuffled[:4378]
y_train = train_label_shuffled[:4378]

x_val = all_features_shuffled[4378:5254]
y_val = train_label_shuffled[4378:5254]

x_test = all_features_shuffled[5254:]
y_test = train_label_shuffled[5254:]

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape,y_test.shape)

(7214, 10)
(7185, 10)
####################################################################################################
####################################################################################################
(3684, 10) (2445, 10)
####################################################################################################
####################################################################################################
(6129,)
1     0
2     0
3     0
11    0
13    0
15    0
17    0
20    0
21    0
27    0
Name: race_num, dtype: int64 
 7184    1
7185    1
7187    1
7188    1
7191    1
7192    1
7194    1
7199    1
7205    1
7206    1
Name: race_num, dtype: int64 0.0 1.0
####################################################################################################
####################################################################################################
test
      c_charge_desc_nan  race_num
1                     0         0
2                    

In [10]:
## Train subset D0/D1
'''
we need split training set to subset 

  D0 = {(y_i, x_i, s_i = 0)} sensitive feature/race_num = 0  African American
  D1 = {(y_i, x_i, s_i = 1)} sensitive feature/race_num = 1  Caucasion

'''
# D0
x_train_black_ind = x_train[:,-1] == 0
x_train_black = x_train[x_train_black_ind]
y_train_black = y_train[x_train_black_ind]

# D1
x_train_white_ind = x_train[:,-1] == 1
x_train_white = x_train[x_train_white_ind]
y_train_white = y_train[x_train_white_ind]


#split out white testors
x_test_white_ind = x_test[:,-1] == 1
x_test_white = x_test[x_test_white_ind]
y_test_white = y_test[x_test_white_ind]

#black testors
x_test_black_ind = x_test[:,-1] == 0
x_test_black = x_test[x_test_black_ind]
y_test_black = y_test[x_test_black_ind]


In [11]:
## x_train stack a all-one column at the beginning of x_train, which will correspond to bias
'''
 x*w + b ->  [1,x] * [b,w]^T
'''
num_training = x_train.shape[0]
one_line_xt = np.ones(num_training).reshape(num_training,1)
x_train1 = np.hstack((one_line_xt, x_train))
print( x_train.shape,x_train1.shape)

## same to black/white D0/D1, subset of train
#split out white/black train set

## train add one column
num_training_white = x_train_white.shape[0]
one_line_xt = np.ones(num_training_white).reshape(num_training_white,1)
x_train_white1 = np.hstack((one_line_xt, x_train_white))
print(x_train_white.shape,x_train_white1.shape)

num_training_black = x_train_black.shape[0]
one_line_xt = np.ones(num_training_black).reshape(num_training_black,1)
x_train_black1 = np.hstack((one_line_xt, x_train_black))
print(x_train_black.shape,x_train_black1.shape)


## test add one column
num_test_white = x_test_white.shape[0]
one_line_xt = np.ones(num_test_white).reshape(num_test_white,1)
x_test_white1 = np.hstack((one_line_xt, x_test_white))
print(x_test_white.shape,x_test_white1.shape)

num_test_black = x_test_black.shape[0]
one_line_xt = np.ones(num_test_black).reshape(num_test_black,1)
x_test_black1 = np.hstack((one_line_xt, x_test_black))
print(x_test_black.shape,x_test_black1.shape)

(4378, 423) (4378, 424)
(1731, 423) (1731, 424)
(2647, 423) (2647, 424)
(367, 423) (367, 424)
(508, 423) (508, 424)


In [12]:
# cvx.log(np.e).value

In [13]:
def BinaryCrossentropyLoss2(x,theta,y_true):
  '''
  x  (n,d+1)
  theta (d+1,1)
  y_true  (n,)
  '''
  num_points = x.shape[0]

  y_true= 2*y_true - 1

  loss = sum( (logistic( multiply(-y_true, x*theta) ))  ) / num_points 

  # loss = sum( multiply(y_true, log(1+exp(-x*theta)) ) )

  return loss

# with no constraints
theta = cvx.Variable(x_train1.shape[1])
# theta.shape (424,)

# initialize a random value of w
np.random.seed(112233)
theta.value = np.random.rand(theta.shape[0])


tau, mu, EPS = 0.5, 1.2, 1e-4 
P1 = cvx.Problem( Minimize( BinaryCrossentropyLoss2( x_train1,theta,y_train ) ),
            [])
             
      
print("problem is DCCP:", dccp.is_dccp(P1))  # true?
print("problem is DCP:", P1.is_dcp())   # false

result = P1.solve(method='dccp', tau=tau, mu=mu, tau_max=1e10,verbose=False)
print("theta including bias" ,theta.value.shape, "comparing to dimension of x_train",x_train.shape[1])

problem is DCCP: True
problem is DCP: True
theta including bias (424,) comparing to dimension of x_train 423


In [14]:
# theta.value

In [15]:
theta_star = theta.value

def predict(x,theta):
  '''
  x   (n,d+1)
  theta (d+1,)

  return y_pred (n,)
  '''
  # l = logit(x,theta)  # (n,)
  
  # y_pred = (logit>0.5) #.astype("float32")

  d = np.dot(x,theta)
  # print(d)
  # print(np.sign(d))
  # print(np.sign(d).sum(),x.shape[0])
  y_pred = (np.sign(d) + 1)/2

  return y_pred

y_pred = predict(x_train1, theta_star)


In [16]:
y_pred.sum()

2088.0

In [17]:
keras.metrics.accuracy(tf.constant(y_train),tf.constant(y_pred)).numpy().mean()

0.9447236

In [23]:
def evaluation(y_true, x, theta):
    y_pred = predict(x,theta)
    CM = confusion_matrix(y_true, y_pred)/len(y_true)
    # print('False positive rate is: ', CM[0][1], 'False negative rate is: ', CM[1][0])
    FPR =  CM[0][1]
    FNR =  CM[1][0]
    return FPR, FNR

## Test
#black accuracy
print("black")
print("False positive rate/","False negative rate ",evaluation(y_test_black, x_test_black1, theta.value))

#white accuracy
print("white")
print("False positive rate/","False negative rate ",evaluation(y_test_white, x_test_white1,theta.value))


black
False positive rate/ False negative rate  (0.05708661417322835, 0.03740157480314961)
white
False positive rate/ False negative rate  (0.043596730245231606, 0.03814713896457766)


In [24]:
## numpy version
def g_theta(y,x,theta):

  '''
  x such as D0 and D1 or D

  x   n*d+1
  y   n*1  (n,)
  theta (d+1,)

  return n*1  (n,)
  '''

  y = 2*y - 1  # (0,1) -> (-1,1)

  d = matmul(x,theta)
  yd = multiply(y,d)

  return minimum(np.zeros_like(yd),yd)


N0 = y_train_black.shape[0]
N1 = y_train_white.shape[0]
N = N0+N1
# print(N,x_train.shape[0]) 4378 4378

def constraint_s0(y_train_black, x_train_black1,theta):

  return N1/N*sum(g_theta(y_train_black,x_train_black1,theta))


def constraint_s1(y_train_white, x_train_white1,theta):

  return N0/N*sum(g_theta(y_train_white,x_train_white1,theta))




In [ ]:
theta2 = Variable(x_train1.shape[1])
# theta.shape (424,)

# initialize a random value of w
np.random.seed(112233)
theta.value = np.random.rand(theta.shape[0])

P1 = cvx.Problem( Minimize( BinaryCrossentropyLoss2( x_train1,theta2,y_train ) ),
            [constraint_s1(y_train_white, x_train_white1,theta2) <= constraint_s0(y_train_black, x_train_black1,theta) + 0.1 , 
            constraint_s1(y_train_white,x_train_white1,theta2) >= constraint_s0(y_train_black, x_train_black1,theta) - 0.1])
             
      
print("problem is DCCP:", dccp.is_dccp(P2))  # true?
print("problem is DCP:", P2.is_dcp())   # false

problem is DCCP: True
problem is DCP: False


In [ ]:
tau, mu, EPS = 0.5, 1.2, 1e-4 
result2 = P2.solve(method='dccp', tau=tau, mu=mu, tau_max=1e10,verbose=True)



ECOS 2.0.7 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -6.888e+03  +4e+04  6e-01  1e+01  1e+00  1e+00    ---    ---    0  0  - |  -  - 
 1  +9.245e-01  -6.798e+03  +4e+04  6e-01  1e+01  1e+00  1e+00  0.1004  9e-01   1  1  1 |  0  0
 2  +2.369e+02  -6.337e+03  +4e+04  2e-01  1e+01  9e-01  1e+00  0.0649  1e+00   1  1  1 |  0  0
 3  +2.094e+02  -5.177e+03  +3e+04  1e-01  1e+01  7e-01  8e-01  0.3088  4e-01   2  1  1 |  0  0
 4  +4.200e+02  -4.758e+03  +3e+04  1e-01  1e+01  7e-01  8e-01  0.0742  9e-01   2  1  1 |  0  0
 5  +3.555e+02  -2.603e+03  +2e+04  7e-02  8e+00  4e-01  4e-01  0.4959  1e-01   2  1  1 |  0  0
 6  +2.866e+02  -2.091e+03  +1e+04  6e-02  7e+00  4e-01  3e-01  0.6578  7e-01   2  1  1 |  0  0
 7  +1.763e+02  -5.222e+02  +4e+03  2e-02  3e+00  1e-01  1e-01  0.7225  6e-02   2  1  1 |  1  1
 8  +4.905e+01  -1.416e+02  +1e+03  1e-02  2e+

In [ ]:
def evaluation(y_true, x, theta):
    y_pred = predict(x,theta)
    CM = confusion_matrix(y_true, y_pred)/len(y_true)
    # print('False positive rate is: ', CM[0][1], 'False negative rate is: ', CM[1][0])
    FPR =  CM[0][1]
    FNR =  CM[1][0]
    return FPR, FNR

## Test
#black accuracy
print("black")
print("False positive rate/","False negative rate ",evaluation(y_test_black, x_test_black1, theta.value))

#white accuracy
print("white")
print("False positive rate/","False negative rate ",evaluation(y_test_white, x_test_white1,theta.value))


black
False positive rate/ False negative rate  (0.045871559633027525, 0.03302752293577982)
white
False positive rate/ False negative rate  (0.03636363636363636, 0.051515151515151514)


In base model:

black\
False positive rate/ False negative rate  (0.05708661417322835, 0.03740157480314961) \
white\
False positive rate/ False negative rate  (0.043596730245231606, 0.03814713896457766)


\

In model with constraints:

black\
False positive rate/ False negative rate  (0.045871559633027525, 0.03302752293577982)\
white\
False positive rate/ False negative rate  (0.03636363636363636, 0.051515151515151514)

Note: training of model with constraints is unstable, it faiils sometimes.